# 1. Download Model

In [ ]:
%cd /content/

!git clone https://github.com/mistralai/mistral-finetune.git

!pip install -r /content/mistral-finetune/requirements.txt

# Download model + set directory
!wget https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar
!DIR=/content/mistral_models && mkdir -p $DIR && tar -xf mistral-7B-v0.3.tar -C $DIR



#_______________Optional___________________________________
# Alternatively, you can download the model from Hugging Face

# !pip install huggingface_hub
# from huggingface_hub import snapshot_download
# from pathlib import Path

# mistral_models_path = Path.home().joinpath('mistral_models', '7B-v0.3')
# mistral_models_path.mkdir(parents=True, exist_ok=True)

# snapshot_download(repo_id="mistralai/Mistral-7B-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)

#! cp -r /root/mistral_models/7B-v0.3 /content/mistral_models
#! rm -r /root/mistral_models/7B-v0.3

!ls /content/mistral_models



# 2. Prepare dataset

In [ ]:
%cd /content/

/content


In [ ]:
# make a new directory called data
!mkdir -p data

In [ ]:

# navigate to this data directory
%cd /content/data

/content/data


In [ ]:

# read data into a pandas dataframe
import pandas as pd

df = pd.read_parquet('https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k/resolve/main/data/test_gen-00000-of-00001-3d4cd8309148a71f.parquet')

# split data into training and evaluation
df_train=df.sample(frac=0.95,random_state=200)
df_eval=df.drop(df_train.index)

# save data into .jsonl files
df_train.to_json("ultrachat_chunk_train.jsonl", orient="records", lines=True)
df_eval.to_json("ultrachat_chunk_eval.jsonl", orient="records", lines=True)


!ls /content/data

ultrachat_chunk_eval.jsonl  ultrachat_chunk_train.jsonl


In [ ]:

# navigate to the mistral-finetune directory
%cd /content/mistral-finetune/

In [ ]:
# some of the training data doesn't have the right format,
# so we need to reformat the data into the correct format and skip the cases that doesn't have the right format:

!python -m utils.reformat_data /content/data/ultrachat_chunk_train.jsonl

In [ ]:
# eval data looking good
!python -m utils.reformat_data /content/data/ultrachat_chunk_eval.jsonl

In [ ]:
# Now you can verify your training yaml to make sure the data is correctly formatted and to get an estimate of your training time.

!python -m utils.validate_data --train_yaml example/7B.yaml


# 3. Start Training

In [ ]:
# these info is needed for training
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"


In [ ]:
# define training configuration
# for your own use cases, you might want to change the data paths, model path, run_dir, and other hyperparameters

config = """
# data
data:
  instruct_data: "/content/data/ultrachat_chunk_train.jsonl"  # Fill
  data: ""  # Optionally fill with pretraining data
  eval_instruct_data: "/content/data/ultrachat_chunk_eval.jsonl"  # Optionally fill

# model
model_id_or_path: "/content/mistral_models"  # Change to downloaded path
lora:
  rank: 64

# optim
# tokens per training steps = batch_size x num_GPUs x seq_len
# we recommend sequence lentgh of 32768
# If you run into memory error, you can try reduce the sequence length
seq_len: 5120
batch_size: 1
num_microbatches: 8
max_steps: 100
optim:
  lr: 1.e-4
  weight_decay: 0.1
  pct_start: 0.05

# other
seed: 0
log_freq: 1
eval_freq: 100
no_eval: False
ckpt_freq: 100

save_adapters: True  # save only trained LoRA adapters. Set to `False` to merge LoRA adapter into the base model and save full fine-tuned model

run_dir: "/content/test_ultra"  # Fill
"""

# save the same file locally into the example.yaml file
import yaml
with open('example.yaml', 'w') as file:
    yaml.dump(yaml.safe_load(config), file)


In [ ]:
! rm -r /content/test_ultra

In [ ]:
# start training

!torchrun --nproc-per-node 1 -m train example.yaml